In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install elasticsearch[async] pandasticsearch
!pip install hashedindex
!apt-get update
!apt-get install tesseract-ocr
!apt-get install tesseract-ocr-por
!pip install tika

In [ ]:
import hashedindex
import tika
import requests
from tika import parser
from hashedindex import textparser


index = hashedindex.HashedIndex()

In [ ]:
topics_rnd3 = pd.read_csv('/kaggle/input/trec-covid-information-retrieval/topics-rnd3.csv')
metadata = pd.read_csv('/kaggle/input/trec-covid-information-retrieval/CORD-19/CORD-19/metadata.csv')

In [ ]:
metadata.head()

In [ ]:
metadata.info()

In [ ]:
topics_rnd3.head()

In [ ]:
topics_rnd3.info()

In [ ]:
subset = pd.DataFrame(metadata, columns = ['cord_uid', 'title', 'abstract', 'pdf_json_files'])
subset_topics = pd.DataFrame(topics_rnd3, columns = ['topic-id', 'query'])

In [ ]:
subset.head()

In [ ]:
subset_topics.head()

In [ ]:
# Start running the tika service
tika.initVM()

final = []
size = 20000

In [ ]:
headers = {
    "X-Tika-OCRLanguage": "eng"
}
def catch_file(data):
    return parser.from_file("/kaggle/input/trec-covid-information-retrieval/CORD-19/CORD-19/" + data["pdf_json_files"], headers=headers)

In [ ]:
for index_t, topic in subset_topics.iterrows():
    for index_m, data in subset.head(size).iterrows():
        if isinstance(data.pdf_json_files, str):
            try:
                text = ""
                conteudo = catch_file(data)
                #print(conteudo.keys())
                for x in conteudo["content"].splitlines():
                    if "\"text\"" in line or "\"title\"" in x:
                        text += x
                
                text = text.replace("\"text\":", "").replace("\"title\":", "").replace("\"", "")
                
                count = text.count(topic.query)
                
                final.append([topic["topic-id"], data["cord_uid"], count]) 
            except:
                continue
#final

In [ ]:
result = pd.DataFrame(final, columns = ['topic-id', 'cord-id', 'count'])
result = result.sort_values(by='topic-id', ascending=True)
result.head(5)

In [ ]:
result[["topic-id","cord-id"]].to_csv('submission.csv',index=False)